# Importation des bibliothèques

In [1]:
import pandas as pd
import sqlite3
import os
import warnings
warnings.filterwarnings("ignore")

# Connexion à la base de données brutes

In [29]:
connect = sqlite3.connect("../Databases/raw-database.db")
cursor = connect.cursor()

# Données de matching des matchs Skill Corner / Stats Bomb

In [3]:
# Lecture de l'unique fichier pour ces données
matching_matches = pd.read_json("Projet_centres_data/matching_matches.json")

In [4]:
# Ecriture des données importées dans une table de la BDD
matching_matches.to_sql("matching_matches", con = connect, if_exists = "replace", index = False)

306

# Données de matching des joueurs Skill Corner / Stats Bomb

In [5]:
# Lecture de l'unique fichier pour ces données
matching_players = pd.read_json("Projet_centres_data/matching_players.json")

In [6]:
# Ecriture des données importées dans une table de la BDD
matching_players.to_sql("matching_players", con = connect, if_exists = "replace", index = False)

639

# Données de matching des équipes Skill Corner / Stats Bomb

In [7]:
# Lecture de l'unique fichier pour ces données
matching_teams = pd.read_json("Projet_centres_data/matching_teams.json")

In [8]:
# Ecriture des données importées dans une table de la BDD
matching_teams.to_sql("matching_teams", con = connect, if_exists = "replace", index = False)

18

# Informations sur les matchs Stats Bomb

In [9]:
# Lecture de l'unique fichier pour ces données
SB_matches = pd.read_json("Projet_centres_data/SB_matches.json")

In [10]:
# Ecriture des données importées dans une table de la BDD
SB_matches.to_sql("SB_matches", con = connect, if_exists = "replace", index = False)

306

# Données events de Stats Bomb

Pour ces données, nous avons un fichier par match, il faut donc importer une liste de fichiers (situés dans le dossier SB_events).  
Pour ce faire, nous allons ouvrir un par un ces fichiers et les concaténer afin d'obtenir un unique jeu de données comprenant l'ensemble des fichiers.  


In [11]:
# Liste des noms des fichiers à ouvrir
liste_fichier_events = os.listdir("Projet_centres_data/SB_events")

In [12]:
# Commande qui permet de concaténer l'ensemble des fichiers
# La boucle for à l'intérieur des [] permet de parcourir la liste des noms des fichiers, et d'ouvrir un fichier un chaque itération
events = pd.concat([pd.read_json("Projet_centres_data/SB_events/" + fichier_event) for fichier_event in liste_fichier_events])

Nous allons maintenant filtrer ces données afin de :  
- Garder uniquement les données dont nous nous servirons. En effet, ces données contiennent 108 variables(colonnes) de base, or elles ne nous sont pas toutes utiles, nous allons donc garder seulement celles qui nous sont utiles afin de réduire la taille des données.  
- Renommer certaines colonnes afin de faciliter notre utilisation de ces dernières.  
- Convertir les colonnes contenant des données stockées sous forme de liste en plusieurs colonnes comprenant chacune un élément de la liste.  
En effet, les bases de données SQLite ne supportent pas les données de type liste, dictionnaire, dataframe etc.

In [13]:
# Liste contenant les noms de colonnes que nous souhaitons garder
colonnes = ["id", "shot_type", "shot_outcome", "type", "match_id", "period", "possession", "location", "pass_cross", "pass_type", "index",
            "pass_end_location", "minute", "shot_end_location", "pass_body_part", "player_id"]

# Dictionnaire permettant de renommer les colonnes souhaitées
dico_rename = {"index" : "index_event", "id" : "event_id", "match_id" : "match_id_SB"}

# Commande pour garder uniquement colonnes souhaitées et renommer les colonnes souhaitées
# De plus, nous utilisons la commande reset_index afin d'obtenir un unique identifiant pour chaque evenement de l'ensemble des données concaténées.
# Nous disposons déjà de la variable event_id qui est unique à chaque event, sauf qu'elle n'est pas "intuitive" et "lisible" car c'est une chaine
# de caractère créée "aléatoirement".
# Ce nouvel index est une range = (0, nombre d'events), et ces events sont triés par ordre chronologique par match dans les données
# concaténées, ce qui est plus intuitif.
# L'event d'index 0 correspondra donc au premier évènement survenu lors du premier match
events = events[colonnes].reset_index(drop = True).rename(dico_rename, axis = 1)

# Les données de la colonne "location" sont des listes de 3 coordonnées, nous allons donc "éclater" cette colonne en 3 colonnes :
# "x_loc", "y_loc" et "z_loc".
# Pour ce faire, nous extrayons d'abord la colonne du dataframe, puis nous créons un autre dataframe comprenant les 3 colonnes des coordonnées
events_loc = events.pop("location").dropna()
events_loc = pd.DataFrame(events_loc.tolist(), index = events_loc.index, columns = ["x_loc", "y_loc", "z_loc"])

# De même pour la colonne "pass_end_location", sauf que celle-ci ne contient que 2 coordonnées
events_pass_loc = events.pop("pass_end_location").dropna()
events_pass_loc = pd.DataFrame(events_pass_loc.tolist(), index = events_pass_loc.index, columns = ["x_pass", "y_pass"])

# De même pour la colonne "shot_end_location" qui comprend 3 coordonnées
events_shot_loc = events.pop("shot_end_location").dropna()
events_shot_loc = pd.DataFrame(events_shot_loc.tolist(), index = events_shot_loc.index, columns = ["x_shot", "y_shot", "z_shot"])

# Il nous reste a concaténer les 3 dataframes créés précédemment avec le dataframe initial.
# Nous concaténons ces dataframes horizontalement (nous les ajoutons à droite du dataframe initial)
events = pd.concat([events, events_loc, events_pass_loc, events_shot_loc], axis = 1)

In [14]:
# Ecriture des données importées dans une table de la BDD
events.to_sql("events", con = connect, if_exists = "replace", index = False)

1118352

# Données freeze frame de Skill Corner sur les centres

De même que pour les données event, nous disposons de 1 fichier par match pour ces données, nous allons donc concaténer l'ensemble des données de chaque fichier dans un seul jeu de données.

In [15]:
# Liste des noms des fichiers à ouvrir
liste_fichier_freeze_frames = os.listdir("Projet_centres_data/SKC_crosses_freeze_frames")

Cependant, cette fois nous souhaitons aussi conserver l'information "match_id" (Skill Corner) pour chaque match, qui n'est pas initialement pas présente dans les données "freeze frames".  
Cette information est en fait stockée dans le nom des fichiers qui sont de la forme "match_id.json".  
Pour ce faire, nous allons, grâce à une boucle for : 
- Parcourir la liste des noms des fichiers à lire (le nom correspond au "match_id" Skill Corner)
- Lire le fichier correspondant
- Ajouter l'information "match_id_SKC" dans une nouvelle colonne (le "match_id" sera ajouté à toutes les lignes du dataframe)
- Concaténer le jeu de données (dataframe) obtenu lors de l'itération au jeu de données comprenant l'ensemble des données des fichier lus lors des précédentes itérations.

In [16]:
# Création du dataframe qui contiendra le jeu de données concaténer (les données de l'ensemble des fichiers de données "freeze frames")
freeze_frames = pd.DataFrame()

# Boucle itérative
for fichier_freeze_frames in liste_fichier_freeze_frames :
    # Lecture du fichier correspondant
    freeze_frames_import = pd.read_json("Projet_centres_data/SKC_crosses_freeze_frames/" + fichier_freeze_frames)
    
    # Création de la colonne "match_id_SKC"
    # Nous enlevons la partie ".json" de la chaine de caractères correspondant au nom du fichier
    freeze_frames_import["match_id_SKC"] = int(fichier_freeze_frames.replace(".json", ""))

    # Concaténation du dataframe obtenu au dataframe comprenenant le jeu de données
    freeze_frames = pd.concat([freeze_frames, freeze_frames_import])

# Pour les mêmes raisons que pour les données events, nous allons créer un nouvel index pour identifier les freeze frames.
freeze_frames.reset_index(drop = True, inplace = True)

# Nous supprimons la variable "image_corners_projection" qui nous est inutile
freeze_frames.drop("image_corners_projection", axis = 1, inplace = True)

Il faut traiter la colonne "possession", qui contient des dictionnaires comprenant chacuns le joueur et l'équipe en possession du ballon au moment ou la frame a été capturée.  
En effet, ces dictionnaires ne peuvent pas être stocké par SQLite...  
Pour ce faire, nous allons procéder comme pour les listes de coordonnées précédentes :
- Supprimer et récupérer la colonne du dataframe
- Eclater cette colonne en 2 nouvelles colonnes qui contiennent respectivement l'équipe en possession du ballon et le joueur en possession du ballon lors de la frame

In [ ]:
freeze_frames_possession = freeze_frames.pop("possession")
freeze_frames[["group", "tackable_object"]] = pd.json_normalize(freeze_frames_possession)

Nous allons maintenant traiter la colonne "data", qui contient pour chaque frame des informations (position, vitesse, etc) pour chaque joueur et le ballon.  
En effet, pour chaque frame, la valeur de cette colonne est en fait une liste de dictionnaire.  
Chaque dictionnaire de la liste correspond aux informations (position, vitesse, etc) pour un joueur ou le ballon.  
De ce fait, les données de cette colonne "data" ne peuvent pas être stockées par SQLite.  
Cependant, nous pouvons voir chaque élèment de cette colonne comme un dataframe, car ces élèments sont des listes de dictionnaires (possédant tous les même clés).  
De ce fait, en convertissant chaque ligne de cette colonne, nous obtiendrons donc (nombre de frames) dataframes.  

In [ ]:
Pour effectuer cette manipulation, nous 

In [17]:
def transform_freeze_frames_data(row) :
    df = pd.DataFrame(row.data)
    df["frame"] = row.frame
    df["match_id_SKC"] = row.match_id_SKC
    return df

In [18]:


freeze_frames_data = pd.concat((freeze_frames.apply(transform_freeze_frames_data, axis = 1)).tolist())
freeze_frames.drop("data", axis = 1, inplace = True)

In [19]:
freeze_frames.to_sql("freeze_frames", con = connect, if_exists = "replace", index = False)

20576

In [20]:
freeze_frames_data.to_sql("freeze_frames_data", con = connect, if_exists = "replace", index = False)

467914

# Données off ball runs de Skill Corner

In [22]:
liste_fichier_off_ball_runs = os.listdir("Projet_centres_data/SKC_off_ball_runs")

In [23]:
off_ball_runs = pd.concat([pd.read_json("Projet_centres_data/SKC_off_ball_runs/" + fichier_off_ball_runs)
                           for fichier_off_ball_runs in liste_fichier_off_ball_runs])

In [24]:
off_ball_runs.to_sql("off_ball_runs", con = connect, if_exists = "replace", index = False)

8489

# Matching pour chaque joueur entre son player_id et son team_id (au sens de Stats Bomb)

In [26]:
liste_fichier_lineups = os.listdir("Projet_centres_data/SB_lineups")

In [27]:
lineups = pd.concat([pd.read_json("Projet_centres_data/SB_lineups/" + fichier_lineups)
                           for fichier_lineups in liste_fichier_lineups])

# Fermeture de la base de données

In [25]:
connect.close()